1. GPU Setup in Colab

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [3]:
!pip install transformers
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

     |████████████████████████████████| 4.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 38.3 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 596 kB 55.0 MB/s 
     |████████████████████████████████| 895 kB 48.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [5]:
data = pd.read_excel('/content/data_Roberta.xlsx')
data1 = data[['phrase','prompt']]
data1.sample(5)

,phrase,prompt
3764,إيران بلد أحبه منذ الصغر ...و باذن الله سأزوه ...,arabe
13258,"خصووووصا إلى كانت درية لي هزاتها , أشمكن يا فا...",darij_arabe
17321,blann makanch frassi 👌👌,darija_latin
12959,توال الكريّفات,darij_arabe
16051,نمشيو نعسو واحد شوية بينما توصل 3:00,darij_arabe


In [6]:
df=data1.copy()
df.isna().sum()

phrase    0
prompt    0
dtype: int64

In [7]:
df['prompt'].value_counts()

darij_arabe     5950
darija_latin    4156
francais        3932
anglais         3677
arabe           3604
Name: prompt, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split

X, sentence_test, y, intent_test = train_test_split(df.phrase, df.prompt, stratify = df.prompt,test_size=0.2, random_state=4612)
sentence_train, sentence_val, intent_train, intent_val = train_test_split(X, y, stratify = y,test_size=0.125, random_state=4612)

In [9]:
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPSILON = 1e-08
EPOCHS = 4
LEARNING_RATE = 2e-5
SEED = 1215
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [10]:
max_len = 0
input = []
length=[]
# For every sentence...
for sent in sentence_train:

    # Tokenize the text and add special tokens--`[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    input.append(input_ids)
    length.append(len(input_ids))
    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))
    mean_len = sum(length)/len(length)
#39 tokens is the maximum number of tokens in a sentence (transcription). Also, a sentence has 14 tokens on average.
print('Max sentence length:%d \nMean sentence length:%d' % (max_len,mean_len))

Token indices sequence length is longer than the specified maximum sequence length for this model (2420 > 512). Running this sequence through the model will result in indexing errors


Max sentence length:5134 
Mean sentence length:52


In [11]:
def tokenize(sentence):
  batch = tokenizer(list(sentence),             
                  is_pretokenized=False,
                  #Pad or truncate all sentences to the same length. Create the attention masks which explicitly differentiate real tokens from [PAD] tokens.
                  padding=True, 
                  truncation=True,
                  return_tensors="pt")
  return batch

In [12]:
tok_train = tokenize(sentence_train)
tok_val = tokenize(sentence_val)
tok_test = tokenize(sentence_test)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword arguments {'is_pretokenized': False} not recognized.
Keyword

In [13]:
from sklearn.preprocessing import LabelEncoder
# encode "intent" to 25 number labels
LE = LabelEncoder()
label_train = torch.tensor((LE.fit_transform(intent_train)))
label_val = torch.tensor((LE.fit_transform(intent_val)))
label_test = torch.tensor((LE.fit_transform(intent_test)))

In [14]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(tok_train['input_ids'], tok_train['attention_mask'],label_train)
validation_dataset = TensorDataset(tok_val['input_ids'], tok_val['attention_mask'],label_val)
test_dataset = TensorDataset(tok_test['input_ids'], tok_test['attention_mask'],label_test)

In [15]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = TRAIN_BATCH_SIZE # Trains with this batch size.
        )

# For validation/test the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            validation_dataset, # The validation samples.
            sampler = SequentialSampler(validation_dataset), # Pull out batches sequentially.
            batch_size = VALID_BATCH_SIZE # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            validation_dataset, 
            sampler = SequentialSampler(validation_dataset), 
            batch_size = VALID_BATCH_SIZE 
        )

In [16]:
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
## use pretained base(relatively small) BERT mdoel for sequence classification 
#CUDA_LAUNCH_BLOCKING=1
model = BertForSequenceClassification.from_pretrained('roberta-base', num_labels = 25)
model.cuda() # make pytorch run this model on GPU.

## use AdamW optimizer
optimizer = AdamW(model.parameters(), 
                  lr = LEARNING_RATE, 
                  eps = EPSILON  )


total_steps = len(train_dataloader) * EPOCHS


scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing BertForSequenceClassification: ['roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.8.attention.output.LayerNorm.weight', 'roberta.encoder.layer.4.attention.output.dense.weight', 'roberta.encoder.layer.9.attention.self.key.weight', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.7.attention.self.key.weight', 'roberta.encoder.layer.6.attention.self.key.bias', 'roberta.encoder.layer.10.attention.self.key.weight', 'roberta.encoder.layer.3.intermediate.dense.weight', 'roberta.encoder.layer.9.intermediate.dense.bias', 'roberta.encoder.layer.1.intermediate.dense.weight', 'roberta.encoder.layer.2.attention.self.query.bias', 'roberta.encoder.layer.7.attention.output.dense.bias', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.8.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.de

In [17]:
def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [18]:
import time
import datetime

def format_time(elapsed):
    #Takes a time in seconds and returns a string hh:mm:ss
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))   
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [19]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/Tensorboard')

In [20]:
# Start the training process:
import random
import torch

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
def train(epochs):
  total_t0 = time.time() # Measure the total training time for the whole run.
  tr_loss = 0
  n_correct = 0
  nb_tr_steps = 0
  nb_tr_examples = 0
  
  # For each epoch...
  for epoch in range(0, epochs):
      print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
      print('Training...')

      t0 = time.time()     # Measure how long the training epoch takes.
      total_tr_loss = 0
      total_n_correct = 0
      total_nb_tr_examples = 0
      model.train()    # Put the model into training mode

      # For each batch of training data...
      for step, batch in enumerate(train_dataloader, 0):     
          # 'batch' contains three pytorch tensors:[0]: input ids, [1]: attention masks, [2]: labels 
          input_ids = batch[0].to(device, dtype = torch.long)
          input_mask = batch[1].to(device, dtype = torch.long)
          labels = batch[2].to(device, dtype = torch.long)

          model.zero_grad()       #clear any previously calculated gradients 

          outputs = model(input_ids, token_type_ids=None, attention_mask=input_mask)
          loss_function = torch.nn.CrossEntropyLoss()
          loss = loss_function(outputs[0], labels) #`loss` is a Tensor containing a single value
          tr_loss += loss.item() #.item()` function just returns the Python value from the tensor
          total_tr_loss += loss.item()
          big_val, big_idx = torch.max(outputs[0], dim=1)
          n_correct += calcuate_accu(big_idx, labels)  
          total_n_correct += calcuate_accu(big_idx, labels)                  
          nb_tr_steps += 1
          nb_tr_examples+=labels.size(0)
          total_nb_tr_examples+=labels.size(0)

          if step % 20==19:
              loss_step = tr_loss/nb_tr_steps
              accu_step = n_correct/nb_tr_examples # #correct examples/all examples 
              print(f"Training Loss per 20 steps(batches): {loss_step}")
              print(f"Training Accuracy per 20 steps(batches): {accu_step}")
              elapsed = format_time(time.time() - t0)    # Calculate elapsed time in minutes.   
              # Report progress.
              print('Batch {} of {}.  Elapsed: {:}.'.format(step+1, len(train_dataloader), elapsed))
              #writer.add_scalar('training loss', loss_step, (epoch +1)*len(trainloader) )
              tr_loss = 0;n_correct = 0;nb_tr_steps = 0;nb_tr_examples = 0
                
          loss.backward() # Perform a backward pass to calculate the gradients.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Clip the norm of the gradients to 1.0. This is to help prevent the "exploding gradients" problem.
          optimizer.step()
          scheduler.step() # Update the learning rate.

    # Calculate the average loss over all of the batches.
      train_loss_per_epoch = total_tr_loss / len(train_dataloader)            
      train_accuracy_per_epoch=total_n_correct/total_nb_tr_examples
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("")
      print("training loss per epoch: {0:.2f}".format(train_loss_per_epoch))
      print("training accuracy per epoch: {0:.2f}".format(train_accuracy_per_epoch))
      print("Training 1 epcoh took: {:}".format(training_time))

In [21]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  488244 KB |  488244 KB |  488244 KB |       0 B  |\n|       from large pool |  487680 KB |  487680 KB |  487680 KB |       0 B  |\n|       from small pool |     564 KB |     564 KB |     564 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |  488244 KB |  488244 KB |  488244 KB |       0 B  |\n|       from large pool |  487680 KB |  487680 KB |

In [22]:
train(epochs = EPOCHS)

======== Epoch 1 / 4 ========
Training...
Training Loss per 20 steps(batches): 2.035527801513672
Training Accuracy per 20 steps(batches): 0.2875
Batch 20 of 1866.  Elapsed: 0:00:28.
Training Loss per 20 steps(batches): 1.5967637360095979
Training Accuracy per 20 steps(batches): 0.35625
Batch 40 of 1866.  Elapsed: 0:00:56.
Training Loss per 20 steps(batches): 1.2981006860733033
Training Accuracy per 20 steps(batches): 0.3875
Batch 60 of 1866.  Elapsed: 0:01:24.
Training Loss per 20 steps(batches): 1.0919921070337295
Training Accuracy per 20 steps(batches): 0.48125
Batch 80 of 1866.  Elapsed: 0:01:53.
Training Loss per 20 steps(batches): 1.1009494185447692
Training Accuracy per 20 steps(batches): 0.40625
Batch 100 of 1866.  Elapsed: 0:02:21.
Training Loss per 20 steps(batches): 0.9138560056686401
Training Accuracy per 20 steps(batches): 0.56875
Batch 120 of 1866.  Elapsed: 0:02:50.
Training Loss per 20 steps(batches): 0.913647148013115
Training Accuracy per 20 steps(batches): 0.53125
Bat

In [23]:
# test the model on the validation set
def valid(model, validation_loader):
  model.eval()
  val_loss = 0
  nb_val_examples = 0
  n_correct = 0
  with torch.no_grad():
    for _, data in enumerate(validation_loader, 0): 
      ids = data[0].to(device, dtype = torch.long)
      mask = data[1].to(device, dtype = torch.long)
      targets = data[2].to(device, dtype = torch.long)
      outputs = model(ids, mask)
      loss_function = torch.nn.CrossEntropyLoss()
      loss = loss_function(outputs[0], targets)
      val_loss += loss.item()
      big_val, big_idx = torch.max(outputs[0], dim=1)
      n_correct += calcuate_accu(big_idx, targets)
      nb_val_examples+=targets.size(0)

  val_ave_loss = val_loss/len(validation_loader)
  val_accu = (n_correct*100)/nb_val_examples
  print("Loss on validation/test data: %0.2f" % val_ave_loss)
  print("Accuracy on validation/test data: %0.2f%%" % val_accu)
  
  return

In [24]:
valid(model, validation_dataloader)

Loss on validation/test data: 0.40
Accuracy on validation/test data: 90.57%


In [25]:
valid(model, test_dataloader)

Loss on validation/test data: 0.40
Accuracy on validation/test data: 90.57%


In [26]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './Documents/languages_detection_Roberta/saved_roberta_model_and_tokenizer/'

# Create output directory if needed
if not os.path.exists(output_dir):
  os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to ./Documents/languages_detection_Roberta/saved_roberta_model_and_tokenizer/


('./Documents/languages_detection_Roberta/saved_roberta_model_and_tokenizer/tokenizer_config.json',
 './Documents/languages_detection_Roberta/saved_roberta_model_and_tokenizer/special_tokens_map.json',
 './Documents/languages_detection_Roberta/saved_roberta_model_and_tokenizer/vocab.json',
 './Documents/languages_detection_Roberta/saved_roberta_model_and_tokenizer/merges.txt',
 './Documents/languages_detection_Roberta/saved_roberta_model_and_tokenizer/added_tokens.json')

In [27]:
df_label = pd.DataFrame(tuple(zip(range(25),LE.classes_)), columns=['id','intent'])
df_label.to_pickle('./Documents/languages_detection_Roberta/saved_roberta_model_and_tokenizer/df_label.pkl')

In [28]:
!cp -r ./Documents/languages_detection_Roberta/saved_bert_model_and_tokenizer/ "gdrive/"

cp: cannot stat './Documents/languages_detection_Roberta/saved_bert_model_and_tokenizer/': No such file or directory


In [39]:
#### load the model and build the detector for deployment
!pip install transformers
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaModel, RobertaTokenizer
input_dir = '/content/Documents/languages_detection_Roberta/saved_roberta_model_and_tokenizer'

loaded_model = BertForSequenceClassification.from_pretrained(input_dir)
loaded_model.eval()
loaded_tokenizer = RobertaTokenizer.from_pretrained(input_dir)
loaded_df_label = pd.read_pickle('/gdrive/languages_detection_Roberta/saved_roberta_model_and_tokenizer/df_label.pkl')

FileNotFoundError: ignored

In [40]:
def language_identification(intent):

  pt_batch = loaded_tokenizer(
  intent,
  padding=True,
  truncation=True,
  return_tensors="pt")

  pt_outputs = loaded_model(**pt_batch)
  __, id = torch.max(pt_outputs[0], dim=1)
  prediction = loaded_df_label.iloc[[id.item()]]['intent'].item()
  print((prediction))
  return 

In [44]:
input ='اسير تلعب' 
language_identification(input)

arabe


In [47]:
torch.save(model.state_dict(), 'L_detection_Mmodel_weights.pth')